In [2]:
search_for = 135
start_from = 0
threads = 10
thread = 6
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.018990516662597656
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 175
all cases: 723027151
type: [1, 1, 1, 1, 135] 135
cases of this type: 332150625
100000 35.69800418396203
200000 56.15648253327129
300000 58.60309670419386
400000 68.20349011372343
500000 73.04878197646887
600000 72.40507288169903
700000 74.81101377599227
800000 71.42301785506167
900000 74.57221496977031
1000000 83.32439614917352
1100000 74.15404552229045
1200000 65.7070428302245
1300000 29.462236759486547
1400000 75.50096320405257
1500000 73.45284113514488
1600000 74.48009454680756
1700000 68.75158580117785
1800000 75.86774417639367
1900000 77.37539024232743
2000000 74.6337546058747
2100000 74.40821059442652
2200000 70.97015751920824
2300000 72.70210232856174
2400000 77.21818611635466
2500000 22.704897627826227
2600000 75.03040506366241
2700000 96.35464014296092
2800000 98.27034742184318
2900000 93.90777062886986
3000000 96.12824288608178


30100000 79.63808894828391
30200000 77.52771043284112
30300000 75.31043350551066
30400000 75.70023491333495
30500000 76.26552845865608
30600000 77.59165530222523
30700000 71.67400243995228
30800000 14.526364759160009
30900000 77.91244881405694
31000000 70.64434025259492
31100000 74.34744162475074
31200000 74.75129733849842
31300000 74.84911830138874
31400000 74.41656826336981
31500000 77.30200665493413
31600000 77.14370498263125
31700000 81.2526838046306
31800000 27.190207724999638
31900000 77.60247116808041
32000000 76.3338456529602
32100000 79.33157107060403
32200000 73.40089950008671
32300000 32.59591787504015
32400000 81.542739980118
32500000 79.33673321139803
32600000 79.98758782570376
32700000 77.69622462028096
32800000 77.54491557026934
32900000 76.11687140727092
33000000 75.8066389599022
33100000 72.46259280742657
33200000 55.50865550813924
33300000 38.23013140574997
33400000 71.46889951535157
33500000 74.13626380902575
33600000 77.15238625000163
33700000 77.33757946569442
3380

60600000 69.02557042183285
60700000 65.20157273551665
60800000 72.20878499787935
60900000 67.5080883429336
61000000 74.75476742677697
61100000 28.95817921325035
61200000 74.66930946142318
61300000 75.23876038895912
61400000 72.09558551806433
61500000 74.25628090914333
61600000 74.27053399385828
61700000 75.14775392291624
61800000 75.9019970028892
61900000 76.73484246401898
62000000 34.84514849454366
62100000 65.78379659766941
62200000 68.47012414950999
62300000 74.88515837600319
62400000 70.68623271494707
62500000 67.78508211027065
62600000 66.93683787586805
62700000 63.21171442398214
62800000 21.34797569482091
62900000 67.21102283683814
63000000 69.00329899832806
63100000 67.91399125669618
63200000 65.49314323005056
63300000 73.37259148605284
63400000 71.7390166576424
63500000 55.179442651796805
63600000 51.01939339998975
63700000 75.31536989541497
63800000 74.45384801085186
63900000 73.50179866961966
64000000 75.81288795066948
64100000 74.34383304123574
64200000 73.2261991072204
6430

90900000 71.550443706526
91000000 63.925254315439155
91100000 69.3010688810137
91200000 68.22761543508041
91300000 74.26010409665471
91400000 74.42178917087466
91500000 68.56253021153228
91600000 70.7382234674657
91700000 56.60131961258578
91800000 35.275000839617235
91900000 67.08153559586091
92000000 63.78706731848604
92100000 69.90315910570108
92200000 65.57234736872529
92300000 19.7125204785117
92400000 62.3282871868022
92500000 65.40989242229453
92600000 63.58965735709964
92700000 65.21946834518407
92800000 50.139345577793605
92900000 40.78970199417587
93000000 65.8862226983511
93100000 68.42744463891646
93200000 72.0106348442033
93300000 72.58752226017701
93400000 65.08058363674222
93500000 72.04892031519833
93600000 68.50768271615948
93700000 66.43897826852782
93800000 69.8092863842365
93900000 70.1234219811157
94000000 65.07938522198619
94100000 68.5243486146413
94200000 37.24128786447012
94300000 44.945527362999485
94400000 60.026018781222206
94500000 62.48515135336014
9460000

120500000 64.68836923818516
120600000 70.29223345144864
120700000 68.23248825681657
120800000 66.5470884613809
120900000 68.18204107543258
121000000 66.1781674173243
121100000 62.37472305922737
121200000 64.03229824928911
121300000 62.76673253960014
121400000 60.67787021842536
121500000 22.673469570953046
121600000 61.55683836544409
121700000 58.89511968518122
121800000 32.96931472037964
121900000 43.7155671139738
122000000 63.52906590063851
122100000 50.27422205704254
122200000 37.34151061484891
122300000 59.253285882910184
122400000 63.317208528172
122500000 59.24115979417831
122600000 59.19510267807022
122700000 60.21307492036663
122800000 67.5968376247497
122900000 67.90965685135507
123000000 69.09863693136725
123100000 64.30731910779421
123200000 69.37089201263575
123300000 66.1740919527056
123400000 68.44008881250227
123500000 65.43991345579897
123600000 65.00583252764466
123700000 57.48768010383083
123800000 61.41353780086352
123900000 60.26870932793541
124000000 20.370965726730

...

639300000 1.8720110905377805
639400000 1.302824985871917
639500000 1.137484886027946
639600000 1.0165547558525814
best so far: 0
type: [1, 1, 3, 5, 9] 135
cases of this type: 7381125
639700000 1.0733284928289413
639800000 0.95163517965191
639900000 1.480210181911875
640000000 1.2636089325390996
640100000 1.2107705748057922
640200000 1.01256039032188
640300000 1.4345935919694344
640400000 1.7778805773511785
640500000 1.3601676485889926
640600000 1.8229070763243853
640700000 1.642071862639708
640800000 1.1196308870946692
640900000 1.7028553684295298
641000000 1.9416213298694722
641100000 1.039615102468903
641200000 0.904128276976017
641300000 0.8772526739776638
641400000 1.4138863036464315
641500000 0.8612577968020558
641600000 0.9929721049992509
641700000 0.9373568545550162
641800000 1.0076775278667212
641900000 1.031399277522724
642000000 1.0884301622738508
642100000 1.3132700753609778
642200000 0.8991650380120647
642300000 0.9877335075225566
642400000 0.8810371463792006
642500000 0.84

666700000 1.6851642807474239
666800000 1.7660958216663127
666900000 1.645618271324579
667000000 2.4666359495338264
667100000 2.248899672105662
667200000 2.7818047239712866
667300000 3.078091092878884
667400000 2.122226755921744
667500000 2.3021129936402325
667600000 3.056222450536706
667700000 1.8201928580420976
best so far: 0
type: [1, 1, 5, 3, 9] 135
cases of this type: 4428675
667800000 2.01271846640408
667900000 0.7392880631308257
668000000 1.1565771199497374
668100000 0.8662887786878222
668200000 1.1788510314707399
668300000 1.3843238976255974
668400000 1.082594366384677
668500000 1.224350022288251
668600000 1.312424354045347
668700000 1.3370441579928185
668800000 1.280062693823254
668900000 1.3868018230352772
669000000 1.2856183536037995
669100000 1.2705352202710392
669200000 1.1384982987562795
669300000 1.3240172878969847
669400000 1.0770217189524194
669500000 0.8682743078547273
669600000 1.1749850458997064
669700000 1.173925997288704
669800000 1.9542491717922688
669900000 1.951

691300000 1.2752389894597114
691400000 1.0699216439953791
691500000 1.0166868442429866
691600000 1.3972195574317277
691700000 0.7785332178866022
691800000 1.2557202408152592
691900000 0.9777093394960165
692000000 0.6573226075452785
692100000 0.8935511952489296
692200000 0.959630099867642
692300000 0.6392082695166289
692400000 1.1335358118655343
692500000 0.8035632418362318
692600000 0.8768801672107233
692700000 1.513081565727204
692800000 0.6250734719255269
692900000 1.0247445698915938
693000000 1.2679181646332192
693100000 1.2730339490650533
693200000 1.091199330540653
693300000 0.9571934728150342
693400000 0.8267753528658568
693500000 0.9556215905065026
693600000 0.7317676587453822
693700000 0.8947347767741303
693800000 1.7013694854865822
693900000 0.7916551157872511
694000000 0.7925419804142654
694100000 0.8231843190179002
694200000 0.9115736162140945
694300000 0.9649842380410194
694400000 1.170472677828706
694500000 1.0898948925799574
694600000 0.6784165870736262
694700000 0.932344

712800000 0.0963164781101452
712900000 0.14492117074977753
best so far: 0
type: [1, 9, 1, 3, 5] 135
cases of this type: 151875
713000000 0.23029386397259233
713100000 0.06830725270936092
best so far: 0
type: [1, 9, 1, 5, 3] 135
cases of this type: 91125
713200000 0.05911127725878954
best so far: 0
type: [1, 9, 1, 15, 1] 135
cases of this type: 30375
best so far: 0
type: [1, 9, 3, 1, 5] 135
cases of this type: 50625
best so far: 0
type: [1, 9, 3, 5, 1] 135
cases of this type: 10125
713300000 0.050840135216735464
best so far: 0
type: [1, 9, 5, 1, 3] 135
cases of this type: 18225
best so far: 0
type: [1, 9, 5, 3, 1] 135
cases of this type: 6075
best so far: 0
type: [1, 9, 15, 1, 1] 135
cases of this type: 2025
best so far: 0
type: [1, 15, 1, 1, 9] 135
cases of this type: 98415
713400000 0.05880801197139024
best so far: 0
type: [1, 15, 1, 3, 3] 135
cases of this type: 32805
best so far: 0
type: [1, 15, 1, 9, 1] 135
cases of this type: 10935
best so far: 0
type: [1, 15, 3, 1, 3] 135
cases o

best so far: 0
type: [5, 9, 3, 1, 1] 135
cases of this type: 81
best so far: 0
type: [5, 27, 1, 1, 1] 135
cases of this type: 27
best so far: 0
type: [9, 1, 1, 1, 15] 135
cases of this type: 50625
best so far: 0
type: [9, 1, 1, 3, 5] 135
cases of this type: 16875
best so far: 0
type: [9, 1, 1, 5, 3] 135
cases of this type: 10125
best so far: 0
type: [9, 1, 1, 15, 1] 135
cases of this type: 3375
723000000 0.0002511757662203577
best so far: 0
type: [9, 1, 3, 1, 5] 135
cases of this type: 5625
best so far: 0
type: [9, 1, 3, 5, 1] 135
cases of this type: 1125
best so far: 0
type: [9, 1, 5, 1, 3] 135
cases of this type: 2025
best so far: 0
type: [9, 1, 5, 3, 1] 135
cases of this type: 675
best so far: 0
type: [9, 1, 15, 1, 1] 135
cases of this type: 225
best so far: 0
type: [9, 3, 1, 1, 5] 135
cases of this type: 1875
best so far: 0
type: [9, 3, 1, 5, 1] 135
cases of this type: 375
best so far: 0
type: [9, 3, 5, 1, 1] 135
cases of this type: 75
best so far: 0
type: [9, 5, 1, 1, 3] 135
cases